In [25]:
!apt install cmake libopenmpi-dev zlib1g-dev
!pip install stable-baselines==2.2.0
!apt-get install python-opengl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.10.2-1ubuntu2).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
libopenmpi-dev is already the newest version (2.1.1-8).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


## Import policy, RL agent, ...

In [0]:
from stable_baselines.common.atari_wrappers import make_atari # Used to obtain the atari environments with Noframeskips
from stable_baselines.deepq.policies import MlpPolicy, CnnPolicy # Policy network based on which decisions are taken
from stable_baselines import DQN # Base DQN library which imlements all the algorithms of Q learning
from stable_baselines.common.vec_env import DummyVecEnv # Transforms the image data to feed to the MLP networks
import gym # Used to import atari game environments


# Imports for exporting the gameplay video 

import time
import matplotlib
#matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.image import imread
from matplotlib import rc, animation
from IPython import display
from IPython.display import HTML
%matplotlib inline

In [0]:
env = gym.make('Atlantis-ram-v0')  # Creates the game environment
env = DummyVecEnv([lambda: env]) # Transforms the data in such a way that it can be fed to the MLPpolicy network

model = DQN(MlpPolicy, env, verbose=1) # Model definition where we define the policy network, environment, and other params
model.learn(total_timesteps=100000) # Model training for some timesteps
model.save("Atlantis") # Saving the model


/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [0]:
frames = [] # Save the gameplay frames
for i in range(1): # Only one gameplay
  obs = env.reset() # Reset environment
  while True: 
      action, _states = model.predict(obs) # returns the states, action, based on current state 
      obs, rewards, dones, info = env.step(action) # takes nect action and returns is the game over, rewards and the next state
      fig = env.render(mode = 'rgb_array') # Render image and save them 
      frames.append(fig)
      if dones == True: # if gameover then exit
        i+=1
        break

In [0]:
# Necessary to convert images to an mp4 file 

!apt install ffmpeg
!which ffmpeg
plt.rcParams['animation.ffmpeg_path'] = u'/usr/bin/ffmpeg'

In [0]:
"""
This cell will compile the frames that should have been saved during training
into an animation.  This required ffmpeg to be installed.

If the main portion wasn't modified, this will have saved frames from every
1,000 episode.  In the animation, you should see it start off performing poorly,
but as it progresses it should perform optimally.
"""

fig, ax = plt.subplots()
plt.axis('off')
l = ax.imshow(frames[0])

def animate(i):
    l.set_data(frames[i])

Writer = animation.writers['ffmpeg']
writer = Writer(fps=12, metadata=dict(artist='Me'))
ani = animation.FuncAnimation(fig, animate, frames=len(frames))

ani.save('Atlantis.mp4', writer=writer, dpi=220)
time.sleep(5) # let it process (only necessary in Colab)

In [0]:
try:
  from google.colab import files
except:
  print("Could not import Google Colab.")
  
# Download atlantis file 
files.download('Atlantis.mp4')